<a href="https://colab.research.google.com/github/fleckc/visSP23/blob/main/ps2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import urllib
import os
from google.colab import files
import time, os, sys, re
import pandas_datareader as pdr
from pandas_datareader import wb

Looking at NJ population, voting habits, and migration. This uses election turnout results from 2000, the 2000 census population counts, and the 2000 census migration patterns in order to retain year similarity. Here are the data sources:

https://www.state.nj.us/state/elections/election-information-ballots-cast.shtml 

(scroll down to 2000 to download the pdf, which I converted to xlsx/csv; it is in a set of cells so it easily converted)

https://www.icpsr.umich.edu/web/ICPSR/studies/13569/

https://www.icpsr.umich.edu/web/ICPSR/studies/13885/




I am interested in seeing the relationships between turnout and population. By population, I mean both the migration and the simple census counts. I do not have a strong idea of what this may look like statistically; it could be as simple as more people move to bigger population areas, but this is why we do statistical analysis.

aok: in my own research i was looking at population change and happiness:
https://theaok.github.io/junk/swbPopGroBrfss_againStudentComSwb.pdf for which i have done vis in Py: https://colab.research.google.com/drive/1fFzDc73LbGAC-G6_I58FV1fH691NAs7_?usp=sharing


aok: and later your could also do sth similar for texas/houston as you know that area

In [2]:
njvote=pd.read_csv('https://github.com/fleckc/visSP23/raw/main/NJ2000ballots.csv')
#aok ok good to name dfs with substantive names

In [3]:
njpop=pd.read_csv('https://github.com/fleckc/visSP23/raw/main/njpop2000.csv')

In [4]:
njmigrate=pd.read_fwf('https://github.com/fleckc/visSP23/raw/main/countymigrationinflow.txt')

In [5]:
njvote.head()

,COUNTY,Total Number of Registered Voters,Total Number of Ballots Cast,Total Ballots Rejected,Percent of Ballots Cast,Total Number of Election Districts
0,Atlantic,135471,93100,398,0.69,157
1,Bergen,492719,369249,582,0.75,554
2,Burlington,239412,178484,897,0.75,349
3,Camden,293752,198565,1271,0.68,326
4,Cape May,68561,48237,163,0.70,124


In [6]:
njvote=njvote.rename(columns={'COUNTY': 'County'})
njvote #renaming for easier merge later

,County,Total Number of Registered Voters,Total Number of Ballots Cast,Total Ballots Rejected,Percent of Ballots Cast,Total Number of Election Districts
0,Atlantic,135471,93100,398,0.69,157
1,Bergen,492719,369249,582,0.75,554
2,Burlington,239412,178484,897,0.75,349
3,Camden,293752,198565,1271,0.68,326
4,Cape May,68561,48237,163,0.70,124
5,Cumberland,76709,49047,98,0.64,93
6,Essex,429546,260478,2596,0.61,587
7,Gloucester,157352,107919,233,0.69,218
8,Hudson,277048,172831,142,0.62,425
9,Hunterdon,74435,56761,88,0.76,107


In [7]:
njpop.head()

,County,"Total Population, 18 years and over","Male Population, 18 years and over","Female Population, 18 years and over","Total Citizens, 18 years and over","Male Citizens, 18 years and over","Female Citizens, 18 years and over","Total Percent citizens, 18 years and over","Male Percent citizens, 18 years and over","Female Percent citizens, 18 years and over"
0,Atlantic County,"188,578","89,044","99,534","174,275","81,855","92,420",92.4,91.9,92.9
1,Bergen County,"681,681","321,102","360,579","587,765","274,953","312,812",86.2,85.6,86.8
2,Burlington County,"317,091","154,961","162,130","307,499","150,499","157,000",97.0,97.1,96.8
3,Camden County,"373,066","175,552","197,514","359,045","168,600","190,445",96.2,96.0,96.4
4,Cape May County,"79,500","37,355","42,145","78,347","36,801","41,546",98.5,98.5,98.6


In [8]:
countiesd = {'County': ['Atlantic', 'Bergen', 'Burlington', 'Camden', 'Cape May', 'Cumberland', 'Essex', 'Gloucester', 'Hudson', 'Hunterdon', 'Mercer', 'Middlesex', 'Monmouth', 'Morris', 'Ocean', 'Passaic', 'Salem', 'Somerset', 'Sussex', 'Union', 'Warren']}
counties = pd.DataFrame(countiesd)
counties #creating a dataframe (from a dict) for the list of counties to "fix" unmatched county name formats (Atlantic vs Atlantic County)
#aok ok fine, but easier just grab it from online say from wikipedia

,County
0,Atlantic
1,Bergen
2,Burlington
3,Camden
4,Cape May
5,Cumberland
6,Essex
7,Gloucester
8,Hudson
9,Hunterdon


In [9]:
njpop=njpop.drop(njpop.index[21:31])
njpop=njpop.drop(['County'], axis=1)
njpop=counties.join(njpop, how="outer")
njpop #dropping extraneous info and renaming counties column for easier merge later
#aok ok good, again very handy and useful to keep it as clean and simple as early as possible

,County,"Total Population, 18 years and over","Male Population, 18 years and over","Female Population, 18 years and over","Total Citizens, 18 years and over","Male Citizens, 18 years and over","Female Citizens, 18 years and over","Total Percent citizens, 18 years and over","Male Percent citizens, 18 years and over","Female Percent citizens, 18 years and over"
0,Atlantic,"188,578","89,044","99,534","174,275","81,855","92,420",92.4,91.9,92.9
1,Bergen,"681,681","321,102","360,579","587,765","274,953","312,812",86.2,85.6,86.8
2,Burlington,"317,091","154,961","162,130","307,499","150,499","157,000",97.0,97.1,96.8
3,Camden,"373,066","175,552","197,514","359,045","168,600","190,445",96.2,96.0,96.4
4,Cape May,"79,500","37,355","42,145","78,347","36,801","41,546",98.5,98.5,98.6
5,Cumberland,"109,258","55,391","53,867","104,561","52,696","51,865",95.7,95.1,96.3
6,Essex,"587,645","271,587","316,058","505,854","229,794","276,060",86.1,84.6,87.3
7,Gloucester,"187,620","88,702","98,918","184,802","87,398","97,404",98.5,98.5,98.5
8,Hudson,"472,046","228,292","243,754","351,622","166,209","185,413",74.5,72.8,76.1
9,Hunterdon,"90,673","44,192","46,481","87,827","42,766","45,061",96.9,96.8,96.9


In [ ]:
njmigrate.head()

,01001,01003,87
0,1001,1005,17
1,1001,1007,7
2,1001,1009,39
3,1001,1013,72
4,1001,1015,56


In [10]:
njmigrate=njmigrate.rename(columns={'01001': 'StateCounty2000',
                 '01003': 'StateCounty1995',
                 '87': 'InflowMigration'})
#renaming columns - note will eliminate some data in this instance but none we will use

In [11]:
njmigratenjonly=njmigrate.query('StateCounty2000 > 34000 & StateCounty2000 < 35000')
#eliminating FIPS codes/counties that are not NJ

In [16]:
njmigratenjonly.head()

,StateCounty2000,StateCounty1995,InflowMigration
408749,34001,1123,27
408750,34001,2020,8
408751,34001,2090,13
408752,34001,2122,4
408753,34001,2240,24


In [12]:
njmigrateonly2=njmigratenjonly.groupby('StateCounty2000')['InflowMigration'].sum()
njmigrateonly2=pd.DataFrame(data=njmigrateonly2)
njmigrateonly2=njmigrateonly2.reset_index(drop="true")
#not really what I wanted but it'll work in the end

In [13]:
njmigrateonly2.head()

,InflowMigration
0,28511
1,89250
2,65805
3,41858
4,17304


In [14]:
njmerge1 = counties.join(njmigrateonly2, how="outer")
#putting county names to the total migration inflows

In [18]:
#aok: in njmigratenjonly you have fips codes, co get dataset with both counties and fipscodes :)
#not only easier but also less dangerous; in your join everything rests on right sorting; much safer to explicitly join on variable!
#eg this has both counties adn fipscodes: https://en.wikipedia.org/wiki/List_of_counties_in_New_Jersey
#and can read from any html straigh up with:
pd.read_html("https://en.wikipedia.org/wiki/List_of_counties_in_New_Jersey", match = "FIPS")[0] #match is keyword search for var in table :)

,County,FIPS code[9],County seat[10],Largest City[11],Est.[10],Formed from[6][7],Named for[12],Density (per mi2),Pop.[13],Area[10],Map
0,Atlantic County,1,Mays Landing,"Egg Harbor Township 47,842",1837,Gloucester County,"The Atlantic Ocean, which forms the county's e...",490.14,274966,"561 sq mi(1,453 km2)",NaN
1,Bergen County,3,Hackensack,"Hackensack 46,030",1683,One of four original counties created in East ...,"Bergen, New Netherland settlement",4076.15,953819,234 sq mi(606 km2),NaN
2,Burlington County,5,Mount Holly,"Evesham Township 46,826",1694,One of two original counties created in West J...,The old ancient name for an inland market near...,576.73,464269,"805 sq mi(2,085 km2)",NaN
3,Camden County,7,Camden,"Cherry Hill 74,553",1844,Gloucester County,"Charles Pratt, 1st Earl Camden (1714–1794), an...",2359.33,523771,222 sq mi(575 km2),NaN
4,Cape May County,9,Cape May Court House,"Lower Township 22,057",1692,Burlington County,The 17th-century Dutch explorer Cornelius Jaco...,375.14,95661,255 sq mi(660 km2),NaN
5,Cumberland County,11,Bridgeton,"Vineland 60,780",1748,Salem County,"Prince William, Duke of Cumberland (1721–1765)...",314.17,153627,"489 sq mi(1,267 km2)",NaN
6,Essex County,13,Newark,"Newark 311,549",1683,One of four original counties created in East ...,The county of Essex in England,6785.06,854917,126 sq mi(326 km2),NaN
7,Gloucester County,15,Woodbury,"Washington Township 48,677",1686,Burlington County,"The city of Gloucester, England",936.85,304477,325 sq mi(842 km2),NaN
8,Hudson County,17,Jersey City,"Jersey City 292,449",1840,Bergen County,"The English explorer Henry Hudson (d. 1611), w...",14946.02,702463,47 sq mi(122 km2),NaN
9,Hunterdon County,19,Flemington,"Raritan Township 23,447",1714,Burlington County,"Robert Hunter (1664–1734), the Colonial Govern...",302.15,129924,"430 sq mi(1,114 km2)",NaN


In [15]:
njmerge1.head()

,County,InflowMigration
0,Atlantic,28511
1,Bergen,89250
2,Burlington,65805
3,Camden,41858
4,Cape May,17304


In [20]:
njmerge2=pd.merge(njmerge1, njvote, how="outer", indicator="merge")
#merging migration and turnout
#aok again when merging turn on the indicator to have a handy _merge variable; and dont forget to investigate that variable as we did in class;
# AND do say explicitly whether it all merge ok, or what are the problems and how you fixed them etc

In [ ]:
njmerge2

,County,InflowMigration,Total Number of Registered Voters,Total Number of Ballots Cast,Total Ballots Rejected,Percent of Ballots Cast,Total Number of Election Districts,merge
0,Atlantic,28511,135471,93100,398,0.69,157,both
1,Bergen,89250,492719,369249,582,0.75,554,both
2,Burlington,65805,239412,178484,897,0.75,349,both
3,Camden,41858,293752,198565,1271,0.68,326,both
4,Cape May,17304,68561,48237,163,0.70,124,both
5,Cumberland,14162,76709,49047,98,0.64,93,both
6,Essex,77576,429546,260478,2596,0.61,587,both
7,Gloucester,29385,157352,107919,233,0.69,218,both
8,Hudson,74744,277048,172831,142,0.62,425,both
9,Hunterdon,27020,74435,56761,88,0.76,107,both


These two datasets merged without issue, as indicated in the merge column. Since there are only 21 observations, we can see this without issue.

In [21]:
njmerge3= pd.merge(njpop, njmerge2, how="outer", on='County', indicator="merge2")
#merging everything together into one dataset with common county naming convention

In [ ]:
njmerge3

The final merge also merged without issue, as indicated by the last two columns. This gives us a dataset with information by county on population, inflow migration, and turnout in the year 2000. This is ready for statisical anaylsis.